In [1]:
from pyspark import SparkContext,SparkConf
from operator import add
from pyspark.sql import SparkSession,Row
from pandas import *
from pyspark.sql.functions import *
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import *
import re
conf = SparkConf().setAppName("pyspark").setMaster("local[4]")
spark = SparkSession.builder\
    .config(conf=conf) \
    .enableHiveSupport()\
    .getOrCreate()
sc=spark.sparkContext

In [14]:
rating_file = 'data/Titanic/train.csv'
from pyspark.sql.types import *
from pyspark.ml import Pipeline,PipelineModel
schema = StructType([
        StructField("PassengerId",IntegerType(), True),
        StructField("Survived",IntegerType(), True),
        StructField("Pclass", IntegerType(), True),
        StructField("Name", StringType(), True),
        StructField("Sex", StringType(), True),
        StructField("Age", IntegerType(), True),
        StructField("Sibsp", IntegerType(), True),
        StructField("Parch", IntegerType(), True),
        StructField("Ticket", StringType(), True),
        StructField("Fare", StringType(), True),
        StructField("Cabin", StringType(), True),
        StructField("Embarked", StringType(), True)])
ratingDf = spark.read.csv(rating_file,schema=schema,header=True)
ratingDf.limit(10).show()
ratingDf
labelInder = StringIndexer(inputCol='label',outputCol='indexedLabel').fit(df)
featureIndex = VectorIndexer(inputCol='features',outputCol='indexedFeatures').fit(df)
(traningDf,testDf) = df.randomSplit(seed=40,weights=[0.7,0.3])
lr = LogisticRegression(featuresCol='indexedFeatures',labelCol='indexedLabel',\
                        maxIter=30,regParam=0.3,elasticNetParam=0.8)
labelConverter = IndexToString(inputCol='prediction',outputCol='predictedLabel',labels=labelInder.labels)
lrPipeline = Pipeline().setStages([labelInder,featureIndex,lr,labelConverter])
lrPipelineModel = lrPipeline.fit(traningDf)
lrPredictions = lrPipelineModel.transform(testDf)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|Sibsp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: int, Sibsp: int, Parch: int, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [11]:
import os
import tensorflow as tf
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
CATEGORICAL_COLUMNS = ["Name", "Sex", "Embarked", "Cabin"]
CONTINUOUS_COLUMNS = ["Age", "SibSp", "Parch", "Fare", "PassengerId", "Pclass"]
SURVIVED_COLUMN = "Survived"
tf.logging.set_verbosity(tf.logging.ERROR)
def DNNLinear(model_dir):
    """Build an estimator."""
    # Categorical columns
    #conversion categorical value into vectors automatically
    #有限集的
    sex = tf.contrib.layers.sparse_column_with_keys(column_name="Sex",keys=["female","male"])
    embarked = tf.contrib.layers.sparse_column_with_keys(column_name="Embarked",keys=["C","S","Q"])
    #类别较多的分类列，无法将所有可能的类别映射为一个整数，所以我们使用哈希值作为键值
    cabin = tf.contrib.layers.sparse_column_with_hash_bucket("Cabin", hash_bucket_size=1000)
    name = tf.contrib.layers.sparse_column_with_hash_bucket("Name", hash_bucket_size=1000)
    #连续列使用的是真实的值
    age = tf.contrib.layers.real_valued_column("Age")
    passenger_id = tf.contrib.layers.real_valued_column("PassengerId")
    sib_sp = tf.contrib.layers.real_valued_column("SibSp")
    parch = tf.contrib.layers.real_valued_column("Parch")
    fare = tf.contrib.layers.real_valued_column("Fare")
    p_class = tf.contrib.layers.real_valued_column("Pclass")
    #桶化（Bucketization ）允许我们找到乘客对应年龄组的生存相关性，而不是将所有年龄作为一个大整体
    age_buckets = tf.contrib.layers.bucketized_column(age,boundaries=[5, 18, 25,30, 35, 40,45, 50, 55, 65])
    #宽列将有效地记住我们与特征之间的交互
    wide_columns = [sex, embarked, cabin, name, age_buckets,
              tf.contrib.layers.crossed_column([age_buckets, sex],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([embarked, name],hash_bucket_size=int(1e4))]
    deep_columns = [
    tf.contrib.layers.embedding_column(sex, dimension=8),
    tf.contrib.layers.embedding_column(embarked, dimension=8),
    tf.contrib.layers.embedding_column(cabin, dimension=8),
    tf.contrib.layers.embedding_column(name, dimension=8),
    age_buckets,
    age,
    passenger_id,
    sib_sp,
    parch,
    fare,
    p_class
    ]

    return tf.contrib.learn.DNNLinearCombinedClassifier(
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[200,50])


def Linear(model_dir):
    """Build an estimator."""
    # Categorical columns
    #conversion categorical value into vectors automatically
    #有限集的
    sex = tf.contrib.layers.sparse_column_with_keys(column_name="Sex",keys=["female","male"])
    embarked = tf.contrib.layers.sparse_column_with_keys(column_name="Embarked",keys=["C","S","Q"])
    #类别较多的分类列，无法将所有可能的类别映射为一个整数，所以我们使用哈希值作为键值
    cabin = tf.contrib.layers.sparse_column_with_hash_bucket("Cabin", hash_bucket_size=1000)
    name = tf.contrib.layers.sparse_column_with_hash_bucket("Name", hash_bucket_size=1000)
    #连续列使用的是真实的值
    age = tf.contrib.layers.real_valued_column("Age")
    passenger_id = tf.contrib.layers.real_valued_column("PassengerId")
    sib_sp = tf.contrib.layers.real_valued_column("SibSp")
    parch = tf.contrib.layers.real_valued_column("Parch")
    fare = tf.contrib.layers.real_valued_column("Fare")
    p_class = tf.contrib.layers.real_valued_column("Pclass")
    #桶化（Bucketization ）允许我们找到乘客对应年龄组的生存相关性，而不是将所有年龄作为一个大整体
    age_buckets = tf.contrib.layers.bucketized_column(age,boundaries=[5, 18, 25,30, 35, 40,45, 50, 55, 65])
    #宽列将有效地记住我们与特征之间的交互
    wide_columns = [sex, embarked, cabin, name, age_buckets,passenger_id,sib_sp,parch,fare,p_class,
              tf.contrib.layers.crossed_column([age_buckets, sex],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([embarked, name],hash_bucket_size=int(1e4))]

    return tf.contrib.learn.LinearClassifier(feature_columns=wide_columns)

def DNN(model_dir):
    """Build an estimator."""
    # Categorical columns
    #conversion categorical value into vectors automatically
    sex = tf.contrib.layers.sparse_column_with_keys(column_name="Sex",keys=["female","male"])
    embarked = tf.contrib.layers.sparse_column_with_keys(column_name="Embarked",keys=["C","S","Q"])
    #类别较多的分类列，无法将所有可能的类别映射为一个整数，所以我们使用哈希值作为键值
    cabin = tf.contrib.layers.sparse_column_with_hash_bucket("Cabin", hash_bucket_size=1000)
    name = tf.contrib.layers.sparse_column_with_hash_bucket("Name", hash_bucket_size=1000)
    #连续列使用的是真实的值
    age = tf.contrib.layers.real_valued_column("Age")
    passenger_id = tf.contrib.layers.real_valued_column("PassengerId")
    sib_sp = tf.contrib.layers.real_valued_column("SibSp")
    parch = tf.contrib.layers.real_valued_column("Parch")
    fare = tf.contrib.layers.real_valued_column("Fare")
    p_class = tf.contrib.layers.real_valued_column("Pclass")
    #桶化（Bucketization ）允许我们找到乘客对应年龄组的生存相关性，而不是将所有年龄作为一个大整体
    age_buckets = tf.contrib.layers.bucketized_column(age,boundaries=[5, 18, 25,30, 35, 40,45, 50, 55, 65])
    #宽列将有效地记住我们与特征之间的交互
    deep_columns = [
    tf.contrib.layers.embedding_column(sex, dimension=8),
    tf.contrib.layers.embedding_column(embarked, dimension=8),
    tf.contrib.layers.embedding_column(cabin, dimension=8),
    tf.contrib.layers.embedding_column(name, dimension=8),
    age_buckets,
    age,
    passenger_id,
    sib_sp,
    parch,
    fare,
    p_class
    ]
    
    return tf.contrib.learn.DNNClassifier(feature_columns=deep_columns,
                                          hidden_units=[100,50,100])
    
def input_fn(df, train=False):
    """Input builder function."""
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)],
                        values=df[k].values,dense_shape=[df[k].size, 1])for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols)
    feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    if train:
        label = tf.constant(df[SURVIVED_COLUMN].values)
          # Returns the feature columns and the label.
        return feature_cols, label
    else:
    # so we can predict our results that don't exist in the csv
        return feature_cols

def train_and_eval(model='DNNLinear'):
    """Train and evaluate the model."""
    df_train = pd.read_csv(
      tf.gfile.Open("./train.csv"),
      skipinitialspace=True)
    df_test = pd.read_csv(
      tf.gfile.Open("./test.csv"),
      skipinitialspace=True)

    model_dir = "./models"
    print "model directory = %s" % model_dir
    
    if model is 'Linear' :
        m=Linear(model_dir)
    elif model is 'DNN' :
        m = DNN(model_dir) 
    else:
        m = DNNLinear(model_dir)         
    m.fit(input_fn=lambda: input_fn(df_train, True), steps=200)
    print "========================================"
    print "model:%s" %model
    r = m.predict(input_fn=lambda: input_fn(df_test))
    s = []
    for i in r:
        s.append(i)
    print "========================================"
    results = m.evaluate(input_fn=lambda: input_fn(df_train, True), steps=1)
    for key in sorted(results):
        print "%s: %s" % (key, results[key])
    df_test["Survived"]=s
    df_test.loc[:,['PassengerId','Survived']].to_csv('./result.csv',index=False)
def main(_):
    train_and_eval('Linear')
    train_and_eval('DNN')
    train_and_eval()
    

if __name__ == "__main__":
    tf.app.run()


model directory = ./models
model:Linear
accuracy: 0.76431
accuracy/baseline_label_mean: 0.383838
accuracy/threshold_0.500000_mean: 0.76431
auc: 0.84769
auc_precision_recall: 0.81222
global_step: 200
labels/actual_label_mean: 0.383838
labels/prediction_mean: 0.207863
loss: 0.808364
precision/positive_threshold_0.500000_mean: 0.854839
recall/positive_threshold_0.500000_mean: 0.464912
model directory = ./models
model:DNN
accuracy: 0.701459
accuracy/baseline_label_mean: 0.383838
accuracy/threshold_0.500000_mean: 0.701459
auc: 0.760032
auc_precision_recall: 0.663499
global_step: 200
labels/actual_label_mean: 0.383838
labels/prediction_mean: 0.381214
loss: 0.567987
precision/positive_threshold_0.500000_mean: 0.682692
recall/positive_threshold_0.500000_mean: 0.415205
model directory = ./models
model:DNNLinear
accuracy: 0.800224
accuracy/baseline_label_mean: 0.383838
accuracy/threshold_0.500000_mean: 0.800224
auc: 0.874993
auc_precision_recall: 0.835183
global_step: 202
labels/actual_label_mea

SystemExit: 

In [37]:
import pandas as pd
import tensorflow as tf
df = pd.read_csv(tf.gfile.Open("./train.csv"),skipinitialspace=True)
continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)],
                    values=df[k].values,dense_shape=[df[k].size, 1])for k in CATEGORICAL_COLUMNS}
# Merges the two dictionaries into one.
feature_cols = dict(continuous_cols)
feature_cols.update(categorical_cols)
feature_cols
# print CATEGORICAL_COLUMNS
# df['Name'].size

{'Age': <tf.Tensor 'Const_78:0' shape=(891,) dtype=int64>,
 'Cabin': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x593cb90>,
 'Embarked': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x5734990>,
 'Fare': <tf.Tensor 'Const_81:0' shape=(891,) dtype=float64>,
 'Name': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x5734890>,
 'Parch': <tf.Tensor 'Const_80:0' shape=(891,) dtype=int64>,
 'PassengerId': <tf.Tensor 'Const_82:0' shape=(891,) dtype=int64>,
 'Pclass': <tf.Tensor 'Const_83:0' shape=(891,) dtype=int64>,
 'Sex': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x5734a10>,
 'SibSp': <tf.Tensor 'Const_79:0' shape=(891,) dtype=int64>}

In [ ]:
import os
import tensorflow as tf
import pandas as pd
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
CATEGORICAL_COLUMNS = ["Name", "Sex", "Embarked", "Cabin"]
CONTINUOUS_COLUMNS = ["Age", "SibSp", "Parch", "Fare", "PassengerId", "Pclass"]
SURVIVED_COLUMN = "Survived"
tf.logging.set_verbosity(tf.logging.ERROR)
def DNNLinear(model_dir):
    """Build an estimator."""
    # Categorical columns
    #conversion categorical value into vectors automatically
    #有限集的
    sex = tf.contrib.layers.sparse_column_with_keys(column_name="Sex",keys=["female","male"])
    embarked = tf.contrib.layers.sparse_column_with_keys(column_name="Embarked",keys=["C","S","Q"])
    #类别较多的分类列，无法将所有可能的类别映射为一个整数，所以我们使用哈希值作为键值
    cabin = tf.contrib.layers.sparse_column_with_hash_bucket("Cabin", hash_bucket_size=1000)
    name = tf.contrib.layers.sparse_column_with_hash_bucket("Name", hash_bucket_size=1000)
    #连续列使用的是真实的值
    age = tf.contrib.layers.real_valued_column("Age")
    passenger_id = tf.contrib.layers.real_valued_column("PassengerId")
    sib_sp = tf.contrib.layers.real_valued_column("SibSp")
    parch = tf.contrib.layers.real_valued_column("Parch")
    fare = tf.contrib.layers.real_valued_column("Fare")
    p_class = tf.contrib.layers.real_valued_column("Pclass")
    #桶化（Bucketization ）允许我们找到乘客对应年龄组的生存相关性，而不是将所有年龄作为一个大整体
    age_buckets = tf.contrib.layers.bucketized_column(age,boundaries=[5, 18, 25,30, 35, 40,45, 50, 55, 65])
    #宽列将有效地记住我们与特征之间的交互
    wide_columns = [sex, embarked, cabin, name, age_buckets,
              tf.contrib.layers.crossed_column([age_buckets, sex],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([embarked, name],hash_bucket_size=int(1e4))]
    deep_columns = [
    tf.contrib.layers.embedding_column(sex, dimension=8),
    tf.contrib.layers.embedding_column(embarked, dimension=8),
    tf.contrib.layers.embedding_column(cabin, dimension=8),
    tf.contrib.layers.embedding_column(name, dimension=8),
    age_buckets,
    age,
    passenger_id,
    sib_sp,
    parch,
    fare,
    p_class
    ]

    return tf.contrib.learn.DNNLinearCombinedClassifier(
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[200,50])


def Linear(model_dir):
    """Build an estimator."""
    # Categorical columns
    #conversion categorical value into vectors automatically
    #有限集的
    sex = tf.contrib.layers.sparse_column_with_keys(column_name="Sex",keys=["female","male"])
    embarked = tf.contrib.layers.sparse_column_with_keys(column_name="Embarked",keys=["C","S","Q"])
    #类别较多的分类列，无法将所有可能的类别映射为一个整数，所以我们使用哈希值作为键值
    cabin = tf.contrib.layers.sparse_column_with_hash_bucket("Cabin", hash_bucket_size=1000)
    name = tf.contrib.layers.sparse_column_with_hash_bucket("Name", hash_bucket_size=1000)
    #连续列使用的是真实的值
    age = tf.contrib.layers.real_valued_column("Age")
    passenger_id = tf.contrib.layers.real_valued_column("PassengerId")
    sib_sp = tf.contrib.layers.real_valued_column("SibSp")
    parch = tf.contrib.layers.real_valued_column("Parch")
    fare = tf.contrib.layers.real_valued_column("Fare")
    p_class = tf.contrib.layers.real_valued_column("Pclass")
    #桶化（Bucketization ）允许我们找到乘客对应年龄组的生存相关性，而不是将所有年龄作为一个大整体
    age_buckets = tf.contrib.layers.bucketized_column(age,boundaries=[5, 18, 25,30, 35, 40,45, 50, 55, 65])
    #宽列将有效地记住我们与特征之间的交互
    wide_columns = [sex, embarked, cabin, name, age_buckets,passenger_id,sib_sp,parch,fare,p_class,
              tf.contrib.layers.crossed_column([age_buckets, sex],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([age_buckets, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, cabin],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([sex, embarked],hash_bucket_size=int(1e6)),
              tf.contrib.layers.crossed_column([embarked, name],hash_bucket_size=int(1e4))]

    return tf.contrib.learn.LinearClassifier(feature_columns=wide_columns)

def input_fn(df, train=False):
    """Input builder function."""
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)],
                        values=df[k].values,dense_shape=[df[k].size, 1])for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols)
    feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    if train:
        label = tf.constant(df[SURVIVED_COLUMN].values)
          # Returns the feature columns and the label.
        return feature_cols, label
    else:
    # so we can predict our results that don't exist in the csv
        return feature_cols

def train_and_eval(model='DNNLinear'):
    """Train and evaluate the model."""
    df_train = pd.read_csv(
      tf.gfile.Open("./train.csv"),
      skipinitialspace=True)
    df_test = pd.read_csv(
      tf.gfile.Open("./test.csv"),
      skipinitialspace=True)

    model_dir = "./models"
    print "model directory = %s" % model_dir
    
    m = DNNLinear(model_dir) if model is 'DNNLinear' else Linear(model_dir)
    m.fit(input_fn=lambda: input_fn(df_train, True), steps=200)
    print "========================================"
    r = m.predict(input_fn=lambda: input_fn(df_test))
    s = []
    for i in r:
        s.append(i)
    print "========================================"
    results = m.evaluate(input_fn=lambda: input_fn(df_train, True), steps=1)
    for key in sorted(results):
        print "%s: %s" % (key, results[key])
    df_test["Survived"]=s
    df_test.loc[:,['PassengerId','Survived']].to_csv('./result.csv',index=False)
def main(_):
    train_and_eval()
    print 'linear model .............'
    train_and_eval('Linear')

if __name__ == "__main__":
    tf.app.run()
